# `metadsl`: seperating API from execution

TODO: Put abstract here with best thing up here.

## Three stories

TODO: Put list of stories here

### Flexibility

TODO: Put viz for laws.

#### "Moore's law is dying"?

[Moore's law](https://en.wikipedia.org/wiki/Moore%27s_law): # transistors doubles every two years

Denard scaling: As transistors get smaller, power stays proportionate to area

> The exponential processor transistor growth predicted by Moore does not always translate into exponentially greater practical CPU performance. Since around 2005–2007, **Dennard scaling appears to have broken down, so even though Moore's law continued for several years after that, it has not yielded dividends in improved performance**.[40][178] The primary reason cited for the breakdown is that at small sizes, current leakage poses greater challenges, and also causes the chip to heat up, which creates a threat of thermal runaway and therefore, further increases energy costs.[40][178]

> The breakdown of Dennard scaling **prompted a switch among some chip manufacturers to a greater focus on multicore processors**, but the gains offered by switching to more cores are lower than the gains that would be achieved had Dennard scaling continued.[179][180] In another departure from Dennard scaling, Intel microprocessors adopted a non-planar tri-gate FinFET at 22 nm in 2012 that is faster and consumes less power than a conventional planar transistor.[181]

Moore's Law + Denard Scaling = [Koomey's Law](https://en.wikipedia.org/wiki/Koomey%27s_law): computations per joule double every 1.57 years

> By the second law of thermodynamics and Landauer's principle, irreversible computing cannot continue to be made more energy efficient forever. As of 2011, computers have a computing efficiency of about 0.00001%.[13] **Assuming that the energy efficiency of computing will continue to double every 1.57 years, the Landauer bound will be reached in 2048**. Thus, after about 2048, Koomey's law can no longer hold. 

How does this relate to writing software in Python?

It means that each CPU instruction will not be run faster. So previous approach to optimization, wait for hardware to get faster, won't work if you are depending on each instruction to happen quicker.

#### Von Neuman Architecture

The architecture of Python, like most current languages, is modeled after the architecture of a CPU. So as CPUs get faster, so does Python.

 We call this sequential access of memory and compute a von Neumann architecture:

[![](https://user-images.githubusercontent.com/1186124/68030190-43d3fd00-fc8f-11e9-8cfd-296d04529f42.png)](https://github.com/Quansight-Labs/metadsl/issues/77#issuecomment-548800090)

> A **von Neumann language is any of those programming languages that are high-level abstract isomorphic copies of von Neumann architectures**.[1] As of 2009, **most current programming languages fit into this description**[citation needed], likely as a consequence of the extensive domination of the von Neumann computer architecture during the past 50 years. 

In [1]:
x = 10
# ...
# fetch arguments
# perform operation
# store result
y = x + x
y

20

#### NumPy

The classic approach of speeding up Python for scientific workloads, shown in tools like NumPy and Pandas, keeps this architecture, but it just optimizes chunks of it. You write specialized optimized instructions for common workloads and call them when we need them. (LaPack, Fortran, C, Cython).

TODO: Put image of chunking

In [2]:
import numpy

x = numpy.arange(1000).reshape(10, 10, 10)
y = x + x
y

array([[[   0,    2,    4,    6,    8,   10,   12,   14,   16,   18],
        [  20,   22,   24,   26,   28,   30,   32,   34,   36,   38],
        [  40,   42,   44,   46,   48,   50,   52,   54,   56,   58],
        [  60,   62,   64,   66,   68,   70,   72,   74,   76,   78],
        [  80,   82,   84,   86,   88,   90,   92,   94,   96,   98],
        [ 100,  102,  104,  106,  108,  110,  112,  114,  116,  118],
        [ 120,  122,  124,  126,  128,  130,  132,  134,  136,  138],
        [ 140,  142,  144,  146,  148,  150,  152,  154,  156,  158],
        [ 160,  162,  164,  166,  168,  170,  172,  174,  176,  178],
        [ 180,  182,  184,  186,  188,  190,  192,  194,  196,  198]],

       [[ 200,  202,  204,  206,  208,  210,  212,  214,  216,  218],
        [ 220,  222,  224,  226,  228,  230,  232,  234,  236,  238],
        [ 240,  242,  244,  246,  248,  250,  252,  254,  256,  258],
        [ 260,  262,  264,  266,  268,  270,  272,  274,  276,  278],
        [ 280,  28



This works great, as long as we can depend on our machines to work roughly like a von Neuman machine.

* Hardware folks can optimize instructions
* software people can try to get their algorithms to run less instructions
* Both approaches will improve performance.

#### End of von Neuman

So if we come back to our earlier point, we see that the von Neuman abstraction starts to leak once we can't improve raw instructions speeds anymore.


List:

We see this in small things like CPU pipelining, SIMD, and memory caching. We also see it in larger changes like GPUs, ASICs, and parallel computing.

None of what I am saying is a new insight. It's just that now as data sizes have increased and our algorithms have gotten more intensive, we need to keep finding ways to increase performance that don't conflict with physics.

John Backus in his 1977 ACM Turing Award lecture (emphasis mine):

> Surely there must be a less primitive way of making big changes in the store than by pushing vast numbers of words back and forth through the von Neumann bottleneck. Not only is this tube a literal bottleneck for the data traffic of a problem, but, more importantly, it is
>
> # an intellectual bottleneck
> ## that has kept us tied to word-at-a-time thinking
> ### instead of encouraging us to think in terms of the larger conceptual units of the task at hand
>
> Thus programming is basically planning and detailing the enormous traffic of words through the von Neumann bottleneck, and much of that traffic concerns not significant data itself, but where to find it.


Again, not a new idea, Haskell & functional programming community have been addressing this. Mainly to orient why these ideas matter for scientific computing.

So what are we talking about? We need to be programming in larger units than just one instruction, if we are to target non-von Neuman machine, which we have to move to to have increased performance.

#### Deep Learning

Deep learning frameworks use larger units than a single instruction.

TODO: Tensorflow graph

Tensorflow came out of distributed system engineering. They build up some large chunk of computations at runtime, then compile them and execute them as a whole. This is precicely the "**larger conceptual units of the task at hand**" that John Backus was talking about all the way back in 1977.

So instead of thinking in sequential terms, we think about "graphs" of computation, or more abstraction representations of what we want. Not as tired to a specific execution strategy.

This allows many different layers to exist for different hardware and different domains. Hardware companies create specialized abstract layers to target their non von Neuman architectures efficiently:

TODO: Give two examples of ecosystem in the graph:

[![](https://github.com/OpenTensors/ecosystem/raw/master/graph.png)](https://github.com/OpenTensors/ecosystem/blob/master/graph.dot)

So we have this tower of non Von Neuman representations of computation. We want to target them from Python, so that users can write code that is **flexible** and **resiliant** with respect to changes in hardware. Just like how you don't have to rewrite your Python code to run it on a seperate CPU. You shouldn't have to rewrite your numeric code, just because you have a new GPU or cluster.

#### Example

Now we will show how `metadsl` can be used to create an abstraction API that is flexible to different backends

In [5]:
from __future__ import annotations

import metadsl
import metadsl_core
import metadsl_visualize

First we design the API, but leave out how it will actually get executed:

In [6]:
class Array(metadsl.Expression):
    @metadsl.expression
    def __getitem__(self, idx: int) -> Array:
        ...
    
    @metadsl.expression
    def __add__(self, other: Array) -> Array:
        ...

@metadsl.expression
def arange(n: int) -> Array:
    ...

What does this look like in von neuman? Build a through line with an example.

In [7]:
def double_and_index(a):
    return (a + a)[5]

In [8]:
double_and_index(arange(10))

Typez(definitions=None, nodes={'-6673148799929700891': ['0', PrimitiveNode(type='int', repr='10')], '4637925139086696156': ['1', CallNode(function='arange', type_params=None, args=['-6673148799929700891'], kwargs=None)], '7589864691381786385': ['2', CallNode(function='Array.__add__', type_params=None, args=['4637925139086696156', '4637925139086696156'], kwargs=None)], '-6673148799917793116': ['3', PrimitiveNode(type='int', repr='5')], '6627396356909184050': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['7589864691381786385', '-6673148799917793116'], kwargs=None)]}, states=States(initial='6627396356909184050', states=None))

In [11]:
import numpy

@expression
def array_from_np(a: numpy.ndarray) -> Array:
    ...

In [10]:
double_and_index(array_from_np(numpy.arange(10)))

Typez(definitions=None, nodes={'-881293237879409691': ['0', PrimitiveNode(type='ndarray', repr='[0 1 2 3 4 5 6 7 8 9]')], '8572533500579682544': ['1', CallNode(function='array_from_np', type_params=None, args=['-881293237879409691'], kwargs=None)], '-5511742583112347613': ['2', CallNode(function='Array.__add__', type_params=None, args=['8572533500579682544', '8572533500579682544'], kwargs=None)], '-6673148799917793116': ['3', PrimitiveNode(type='int', repr='5')], '-4475861119879165229': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['-5511742583112347613', '-6673148799917793116'], kwargs=None)]}, states=States(initial='-4475861119879165229', states=None))

In [12]:
@register
@rule
def add_np_arrays(l: numpy.ndarray, r: numpy.ndarray) -> R[Array]:
    return array_from_np(l) + array_from_np(r), lambda: array_from_np(l + r)


In [14]:
@register
@rule
def getitem_np_arrays(a: numpy.ndarray, i: int) -> R[Array]:
    return array_from_np(a)[i], lambda: array_from_np(numpy.asarray(a[i]))

How did we get here?

In [15]:
double_and_index(array_from_np(numpy.arange(10)))

Typez(definitions=None, nodes={'-881295820403010891': ['0', PrimitiveNode(type='ndarray', repr='[0 1 2 3 4 5 6 7 8 9]')], '-8133998376571335146': ['1', CallNode(function='array_from_np', type_params=None, args=['-881295820403010891'], kwargs=None)], '-1543648475558051165': ['2', CallNode(function='Array.__add__', type_params=None, args=['-8133998376571335146', '-8133998376571335146'], kwargs=None)], '-6673148799917793116': ['3', PrimitiveNode(type='int', repr='5')], '-3611070570820362031': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['-1543648475558051165', '-6673148799917793116'], kwargs=None)], '-881297224411955291': ['5', PrimitiveNode(type='ndarray', repr='[ 0  2  4  6  8 10 12 14 16 18]')], '-5055665085418818458': ['2', CallNode(function='array_from_np', type_params=None, args=['-881297224411955291'], kwargs=None)], '-6171199438712838398': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['-5055665085418818458', '-6673148799917793116'], k

But what about a symbolic execution?

In [16]:
@expression
def array_from_string(a: str) -> Array:
    ...

In [17]:
@register
@rule
def add_str_arrays(l: str, r: str) -> R[Array]:
    return array_from_string(l) + array_from_string(r), lambda: array_from_string(f"({l} + {r})")

@register
@rule
def getitem_str_arrays(a: str, i: int) -> R[Array]:
    return array_from_string(a)[i], lambda: array_from_string(f"{a}[{i}]")

Talk about other code generation tools like numba and sympy. How does it relate?

In [18]:
double_and_index(array_from_string("a"))

Typez(definitions=None, nodes={'284729093971230013': ['0', PrimitiveNode(type='str', repr='a')], '4229298937763597194': ['1', CallNode(function='array_from_string', type_params=None, args=['284729093971230013'], kwargs=None)], '-455434853523016285': ['2', CallNode(function='Array.__add__', type_params=None, args=['4229298937763597194', '4229298937763597194'], kwargs=None)], '-6673148799917793116': ['3', PrimitiveNode(type='int', repr='5')], '872318015315686865': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['-455434853523016285', '-6673148799917793116'], kwargs=None)], '1268004110435074158': ['5', PrimitiveNode(type='str', repr='(a + a)')], '-7374487091250280579': ['2', CallNode(function='array_from_string', type_params=None, args=['1268004110435074158'], kwargs=None)], '-4842764275136866916': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['-7374487091250280579', '-6673148799917793116'], kwargs=None)], '-5386868795135032884': ['6', Primitive

So what we are creating here is a way to define interface for our computation in Python, and then different ways of executing it by means of evaluation.

Hold on...  What's going on here? Graphs? Expressions? Rules? What's all this confusing junk?

Let's take a step back. I said we have all these non Von Neuman architectures we want to represent as we are programming. Does that mean we all need to leave Python and switch to Julia or Swift or Haskell since they are so much smarter and can compile things quicker?

"But I like Python!", you might say. "It's fun and inviting and has a nice community. Plus we have a lot of sunk infastructure costs!"

TODO: Computing for everybody proposal

TODO: Logos for haskell and julia and python

I agree! And so do the deep learning frameworks, for the most part. That's why they all have Python APIs. So what we are looking for is not really a new language, but a way to express what we want to compute in Python in a way that we can later target it to other hardware.

#### "Folding Domain-Specific Languages: Deep and Shallow Embeddings" - Jeremy Gibbons


We call this a domain specific language.

> There are two main approaches to DSLs. **Standalone DSLs** provide their own custom syntax and semantics, and standard compilation techniques are used to translate or interpret programs written in the DSL for execution. Standalone DSLs can be designed for maximal convenience to their intended users. But **the exercise can be a significant undertaking for the implementer**, involving an entirely separate ecosystem—compiler, editor, debugger, and so on—and typically also much reinven- tion of standard language features such as variables, definitions, and conditionals.

> The alternative approach is to **embed the DSL within a host GPL**, essentially as a collection of definitions written in the host language. **All the existing facilities and infrastructure of the host environment can be appropriated for the DSL**, and familiarity with the syntactic conventions and tools for the host language can be carried over to the DSL. Whereas the standalone approach is the most common one within object-oriented circles [5], the embedded approach is typically favoured by functional programmers [11]. It seems that core FP features such as algebraic datatypes and higher-order functions are extremely helpful in defining embedded DSLs; conversely, it has been said that language-oriented tasks such as DSLs are the killer application for FP.

> Amongst embedded DSLs, there are two further refinements. With **a deep embedding, terms in the DSL are implemented simply to construct an abstract syntax tree (AST)**; this tree is subsequently transformed for optimization and traversed for evaluation. With **a shallow embedding, terms in the DSL are implemented directly as the values to which they evaluate, bypassing the intermediate AST and its traversal**. The deep and shallow embeddings are closely related, and intimately connected to folds; the purpose of this paper is to explore that connection.

TODO: Figure out how to pair down the quotes. Tell what's important.

NumPy is a shallowly embedded domain specific language, since the "terms" (functions) in it are executed immidiatly. No AST is created.

`metadsl` lets you create APIs like NumPy that are deeply embedded, so that first we construct an AST. That's what we saw in the images above. Then we can transform that tree to either evaluate it direcly or compile it to another form.

So maybe it really should be `metadedsl` (meta deeply embedded domain specific language)?

Metadsl allows creating a deeply embedding languages in Python to optimize and compile them.

### Optimization

TODO: Say what we can optimize with this tool and how it compares to other optimization work:

In [19]:
@register_pre
@rule
def optimize_getitem_add(l: Array, r: Array, idx: int) -> R[Array]:
    return (
        # expression to match against
        (l + r)[idx],
        # expression to replace it with
        l[idx] + r[idx]
    )

TODO: Open old output

TODO: Use terms from API.

In [21]:
double_and_index(array_from_string("a"))

Typez(definitions=None, nodes={'284729093971230013': ['5', PrimitiveNode(type='str', repr='a')], '4229298937763597194': ['6', CallNode(function='array_from_string', type_params=None, args=['284729093971230013'], kwargs=None)], '-455434853523016285': ['2', CallNode(function='Array.__add__', type_params=None, args=['4229298937763597194', '4229298937763597194'], kwargs=None)], '-6673148799917793116': ['7', PrimitiveNode(type='int', repr='5')], '872318015315686865': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['-455434853523016285', '-6673148799917793116'], kwargs=None)], '-4474825845148502855': ['8', CallNode(function='Array.__getitem__', type_params=None, args=['4229298937763597194', '-6673148799917793116'], kwargs=None)], '7533663746743132173': ['4', CallNode(function='Array.__add__', type_params=None, args=['-4474825845148502855', '-4474825845148502855'], kwargs=None)], '3816845342069196749': ['9', PrimitiveNode(type='str', repr='a[5]')], '-288570790305451619': 

Mathematics of arrays... Show this optimization as well...

In [22]:
def array_from_function(indxs: )

SyntaxError: invalid syntax (<ipython-input-22-9c325d562f19>, line 1)

### Introspection

We can start to think about semantic open research and connection with opening. Reproducible science. Makes compute data so we can understand our scientific processes better.

Benefits scientists

TODO: Shoot for 1 minute a cell

Down to 40 cells, make them a little more complex

What do I want people to take away?

* Fits within Python ecosystem and tooling (feels nice).
* Encourages best practices.

Should be fun (to give and to watch)! No one wants to listen to a talk that the presenter doesn't wanna give.

Talk about challenges! 

What do I want? I want to know the problems, who needs help. This is hard!

Issues, vendor lock, where can people help. List of current directions its moving in, graph of this?

## Current State

#### LLVM

In [25]:
import metadsl as m
import metadsl_core as mc
import metadsl_llvm as ml
import metadsl_visualize

In [26]:
def create_metadsl_fn():

    ##
    # Constants
    ##

    int_type = ml.Type.create_int(32)
    zero = ml.Value.constant(int_type, 0)
    one = ml.Value.constant(int_type, 1)

    ##
    # Module Reference
    ##
    mod = ml.ModuleReference.create("fib")

    ##
    # Module Builder
    ##
    mod_builer = ml.ModuleBuilder.create(mod)

    ##
    # Function References
    ##
    mod_builder, fib_more_fn = ml.FunctionReference.create(
        mod_builer,
        ml.FunctionType.create(int_type, int_type, int_type, int_type),
        "fib_more",
        "fastcc",
    ).spread

    mod_builder, fib_fn = ml.FunctionReference.create(
        mod_builer, ml.FunctionType.create(int_type, int_type), "fib", "fastcc"
    ).spread

    ##
    # Function Builders
    ##
    fib_more_builder = ml.FunctionBuilder.create(fib_more_fn)
    fib_fn_builder = ml.FunctionBuilder.create(fib_fn)

    ##
    # Arguments
    ##

    fib_n = fib_fn_builder.arguments[mc.Integer.from_int(0)]
    fib_more_n = fib_more_builder.arguments[mc.Integer.from_int(0)]
    fib_more_a = fib_more_builder.arguments[mc.Integer.from_int(1)]
    fib_more_b = fib_more_builder.arguments[mc.Integer.from_int(2)]

    ##
    # Block References
    ##
    fib_fn_builder, fib_entry = ml.BlockReference.create("entry", fib_fn_builder).spread
    fib_more_builder, fib_more_entry = ml.BlockReference.create(
        "entry", fib_more_builder
    ).spread
    fib_more_builder, fib_pred_cont = ml.BlockReference.create(
        "pred_cont", fib_more_builder
    ).spread
    fib_more_builder, fib_not_pred_cont = ml.BlockReference.create(
        "not_pred_cont", fib_more_builder
    ).spread
    fib_more_builder, fib_n_eq_one = ml.BlockReference.create(
        "n_eq_one", fib_more_builder
    ).spread
    fib_more_builder, fib_n_neq_one = ml.BlockReference.create(
        "n_neq_one", fib_more_builder
    ).spread

    ##
    # Block Builders
    ##
    fib_entry_builder = ml.BlockBuilder.create(fib_entry)
    fib_more_entry_builder = ml.BlockBuilder.create(fib_more_entry)
    fib_pred_cont_builder = ml.BlockBuilder.create(fib_pred_cont)
    fib_not_pred_cont_builder = ml.BlockBuilder.create(fib_not_pred_cont)
    fib_n_eq_one_builder = ml.BlockBuilder.create(fib_n_eq_one)
    fib_n_neq_one_builder = ml.BlockBuilder.create(fib_n_neq_one)

    fib_entry_builder, fib_entry_res = fib_entry_builder.call(
        fib_more_fn, mc.Vec.create(fib_n, zero, one)
    ).spread
    fib_entry_builder = fib_entry_builder.ret(fib_entry_res)

    fib_more_entry_builder, pred_cont = fib_more_entry_builder.icmp_signed(
        ">", fib_more_n, one
    ).spread

    fib_more_entry_builder = fib_more_entry_builder.cbranch(
        pred_cont, fib_pred_cont, fib_not_pred_cont
    )

    fib_pred_cont_builder, minus1 = fib_pred_cont_builder.sub(fib_more_n, one).spread
    fib_pred_cont_builder, ab = fib_pred_cont_builder.add(fib_more_a, fib_more_b).spread

    fib_pred_cont_builder, added = fib_pred_cont_builder.call(
        fib_more_fn, mc.Vec.create(minus1, fib_more_b, ab)
    ).spread
    fib_pred_cont_builder = fib_pred_cont_builder.ret(added)

    fib_not_pred_cont_builder, n_eq_1 = fib_not_pred_cont_builder.icmp_signed(
        "==", fib_more_n, one
    ).spread
    fib_not_pred_cont_builder = fib_not_pred_cont_builder.cbranch(
        n_eq_1, fib_n_eq_one, fib_n_neq_one
    )

    fib_n_eq_one_builder = fib_n_eq_one_builder.ret(fib_more_b)

    fib_n_neq_one_builder = fib_n_neq_one_builder.ret(fib_more_a)

    ##
    # Blocks
    ##
    fib_entry_block = ml.Block.create(fib_entry, fib_entry_builder)
    fib_more_entry_block = ml.Block.create(fib_more_entry, fib_more_entry_builder)
    fib_pred_cont_block = ml.Block.create(fib_pred_cont, fib_pred_cont_builder)
    fib_not_pred_cont_block = ml.Block.create(
        fib_not_pred_cont, fib_not_pred_cont_builder
    )
    fib_n_eq_one_block = ml.Block.create(fib_n_eq_one, fib_n_eq_one_builder)
    fib_n_neq_one_block = ml.Block.create(fib_n_neq_one, fib_n_neq_one_builder)

    ##
    # Functions
    ##
    fib_fn_real = ml.Function.create(fib_fn, mc.Vec.create(fib_entry_block))
    fib_more_fn_real = ml.Function.create(
        fib_more_fn,
        mc.Vec.create(
            fib_more_entry_block,
            fib_pred_cont_block,
            fib_not_pred_cont_block,
            fib_n_eq_one_block,
            fib_n_neq_one_block,
        ),
    )

    ##
    # Module
    ##

    module_real = ml.Module.create(mod, mc.Vec.create(fib_fn_real, fib_more_fn_real))

    ##
    # CType
    ##
    c_int = ml.CType.c_int()
    c_func_type = ml.CFunctionType.create(c_int, c_int)

    return ml.compile_function(module_real, mod_builder, fib_fn, c_func_type)


In [28]:
metadsl_fn = m.execute(create_metadsl_fn())

Typez(definitions=None, nodes={'-2084651250070441828': ['272', PrimitiveNode(type='str', repr='fib')], '8611334174738714696': ['198', CallNode(function='ModuleReference.create', type_params=None, args=['-2084651250070441828'], kwargs=None)], '-5275260287871924045': ['104', CallNode(function='ModuleBuilder.create', type_params=None, args=['8611334174738714696'], kwargs=None)], '-6673148799957846541': ['105', PrimitiveNode(type='int', repr='32')], '7704737798639857470': ['106', CallNode(function='Type.create_int', type_params=None, args=['-6673148799957846541'], kwargs=None)], '3041189459414255294': ['107', CallNode(function='FunctionType.create', type_params=None, args=['7704737798639857470', '7704737798639857470'], kwargs=None)], '6596398620974637448': ['108', PrimitiveNode(type='str', repr='fastcc')], '-3347690120971598309': ['109', CallNode(function='FunctionReference.create', type_params=None, args=['-5275260287871924045', '3041189459414255294', '-2084651250070441828', '659639862097

In [19]:
metadsl_fn(1000)

1556111435

#### Next Steps

Need use case!